In [4]:
import pypyodbc
import pandas as pd
import numpy as np
from math import *
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
from scipy.spatial.distance import cdist

from scipy.spatial.distance import squareform
from scipy.spatial.distance import pdist, jaccard
from sqlalchemy import create_engine

#### Read Data from Files

In [5]:
movie_id = "imdb_id"
author_username = "author"
ratings = "stars"

train_df = pd.read_csv("training_set.csv")
test_df = pd.read_csv("testing_set.csv")

In [6]:
train_df

,date,imdb_id,author,stars
0,2022-07-21,tt0765010,nicktusk-95591,7
1,2022-05-01,tt0765010,peterwixongb,7
2,2022-01-19,tt0765010,kamsitheprince,9
3,2021-09-08,tt0765010,zacharyrivas21,7
4,2021-08-11,tt0765010,SameirAli,7
...,...,...,...,...
365643,2006-12-28,tt0454082,superduperspit,5
365644,2006-12-26,tt0454082,FrightMeter,9
365645,2006-12-25,tt0454082,sackjigler,7
365646,2006-12-25,tt0454082,Nightman85,3


In [7]:
# Time Calc

#### Build matrix from users and movies

In [8]:
df_matrix = train_df.pivot_table(index=author_username, columns=movie_id, values=ratings)
df_matrix.reset_index(inplace=True)

# Rút gọn còn 1000 user để test
df_matrix = df_matrix[:1000]
df_matrix

imdb_id,author,tt0000417,tt0010323,tt0013442,tt0015648,tt0015864,tt0017136,tt0020629,tt0021749,tt0021814,...,tt9772374,tt9777666,tt9783600,tt9784456,tt9784798,tt9794630,tt9844522,tt9845564,tt9848626,tt9907782
0,007Waffles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00Yasser,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,04GreatFlick,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0maro0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0w0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Farzad-Doosti,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
996,FatMan-QaTFM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
997,FattyBoomBatty,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
998,FeastMode,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# Time Calc

#### Matrix Calculation

In [10]:
def calc_cosine_similarity(arr):
    # Caclculate euclidean distance between users, perform parallelization here
    euclidean_distance_matrix = []
    for i in arr:
        user_distance = []
        for j in arr:
            vector_distance = i - j
            euclidean_calc = 0
            for ele in vector_distance:
                if isnan(ele) == False:
                    euclidean_calc += ele*ele
            user_distance.append(sqrt(euclidean_calc))
        euclidean_distance_matrix.append(user_distance)
    # Calculate cosine similarity
    euclidean_distance_matrix = np.array(euclidean_distance_matrix).T
    A_sparse = sparse.csr_matrix(euclidean_distance_matrix)
    cosine_similarities = cosine_similarity(A_sparse)

    return cosine_similarities

In [11]:
test_arr = calc_cosine_similarity(np.array(df_matrix.iloc[:,1:]))

In [ ]:
# Time Calc

In [12]:
user_cosine_similarity = pd.DataFrame(data=test_arr,index=list(df_matrix[author_username]),columns=list(df_matrix[author_username]))
user_cosine_similarity

,007Waffles,00Yasser,04GreatFlick,0maro0,0w0,108YearsOld,109YearsOld,10sion,121mcv,13Funbags,...,FallenEye,Fallenhazel,FallsDownz,FandomFanatic21,Faristuta,Farzad-Doosti,FatMan-QaTFM,FattyBoomBatty,FeastMode,Feel-the-truth
007Waffles,1.000000,0.346984,0.298321,0.525135,0.542194,0.499928,0.539461,0.298269,0.238517,0.218292,...,0.532436,0.096788,0.283915,0.328332,0.495022,0.443352,0.262453,0.548754,0.525805,0.527876
00Yasser,0.346984,1.000000,0.086213,0.274337,0.431617,0.218397,0.155002,0.422531,0.111071,0.255042,...,0.384711,0.188673,0.250300,0.264460,0.361793,0.383920,0.234919,0.377100,0.424072,0.304166
04GreatFlick,0.298321,0.086213,1.000000,0.099290,0.151134,0.087695,0.203956,0.049861,0.076832,0.039455,...,0.136815,0.019153,0.014937,0.066025,0.204296,0.151518,0.051837,0.097185,0.141995,0.167211
0maro0,0.525135,0.274337,0.099290,1.000000,0.508081,0.394769,0.373947,0.328507,0.080608,0.253919,...,0.476631,0.117980,0.237520,0.353755,0.369462,0.329594,0.308251,0.549877,0.499983,0.396691
0w0,0.542194,0.431617,0.151134,0.508081,1.000000,0.436213,0.265203,0.646984,0.147164,0.384080,...,0.752121,0.251302,0.437168,0.472611,0.491372,0.528550,0.410372,0.477065,0.786068,0.488588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Farzad-Doosti,0.443352,0.383920,0.151518,0.329594,0.528550,0.353974,0.228761,0.525080,0.171012,0.282625,...,0.447172,0.224045,0.313303,0.273976,0.342757,1.000000,0.499574,0.337074,0.560237,0.301854
FatMan-QaTFM,0.262453,0.234919,0.051837,0.308251,0.410372,0.298770,0.133574,0.400379,0.040025,0.298456,...,0.340115,0.292653,0.267133,0.226746,0.206890,0.499574,1.000000,0.275361,0.526522,0.156300
FattyBoomBatty,0.548754,0.377100,0.097185,0.549877,0.477065,0.422036,0.312087,0.293309,0.094412,0.258691,...,0.487623,0.089018,0.293692,0.358466,0.392293,0.337074,0.275361,1.000000,0.473930,0.410705
FeastMode,0.525805,0.424072,0.141995,0.499983,0.786068,0.486637,0.293471,0.701135,0.123897,0.477344,...,0.722708,0.312644,0.428257,0.446255,0.480385,0.560237,0.526522,0.473930,1.000000,0.450352


Xây dựng tập user Neighborhood

In [13]:
# Construct Neighborhood
# Chọn ra top 100 users có độ tương tự cao nhất là các neighbors
#threshold = 0.95
K = 100
neighborhood_combine = {}
for user in user_cosine_similarity.columns:
    #neighborhood = list(user_cosine_similarity[user][(user_cosine_similarity[user] > threshold) & (user_cosine_similarity[user] < user_cosine_similarity[user].max())].index)
    neighborhood = list(user_cosine_similarity[user][(user_cosine_similarity[user] < user_cosine_similarity[user].max())].drop_duplicates(keep='first').nlargest(K).index)
    neighborhood_combine[user] = neighborhood

In [14]:
# Construct Neighborhood with expanded k-RNN
# Kiểm tra đặc trưng Neighborhood lẫn nhau giữa một user so với các neighbors của chúng
gamma = 1
neighborhood_list = []
neighborhood_combine_1 = neighborhood_combine.copy()
for key,value in neighborhood_combine.items():
    for v in value:
        if key in neighborhood_combine[v]:
            continue
        else:
            neighborhood_combine[key].remove(v)

In [20]:
user_id = '007Waffles'

service_used_by_user_id = df_matrix[df_matrix[author_username] == user_id].dropna(axis=1, how='all')
similar_services = df_matrix[df_matrix[author_username].isin(neighborhood_combine[user_id])].dropna(axis=1, how='all')
similar_services.drop(service_used_by_user_id.columns[1:],axis=1, inplace=True, errors='ignore')
item_score = {}

# Loop through items
for i in similar_services.columns[1:]:
  # Create a variable to store the score
  total = 0
  # Create a variable to store the number of scores
  count = 0
  # Loop through similar users
  for u in neighborhood_combine[user_id]:
    if isnan(similar_services[i][similar_services[author_username] == u].iloc[0]) is False:
    # Score is the sum of user similarity score
      score = user_cosine_similarity[user_id][u] * similar_services[i][similar_services[author_username] == u].values[0]
      total += score
      count +=1

  item_score[i] = total #/ count
# Convert dictionary to pandas dataframe
item_score = pd.DataFrame(item_score.items(), columns=['movie', 'movie_score'])
item_score['count'] = list(similar_services.count().values)[1:]
# Sort the services by score
ranked_item_score = item_score.sort_values(by='movie_score', ascending=False)

# Select top m services
m = 10
print("Top 10 bộ phim được khuyến khích cho user",user_id)
ranked_item_score[['movie','movie_score']].head(m)

Top 10 bộ phim được khuyến khích cho user 007Waffles


,movie,movie_score
559,tt2382320,59.376279
573,tt2527338,56.607708
762,tt6264654,55.820094
650,tt4154796,50.676687
807,tt7131622,49.556791
659,tt4244994,49.455806
648,tt4154664,47.754708
449,tt1477834,46.217473
806,tt7126948,42.463652
897,tt9376612,42.440031


In [18]:
neighborhood_combine[user_id]

['Draysan-Jennings',
 'Exorcistus',
 'DoNotComeToTheCinemaDepressed',
 'AGood',
 'Eli_Elvis',
 'DongDiggity',
 'Ckn48',
 'ChrisInMiami',
 'Darwinskid',
 'DuskShadow',
 'Erik_Stone',
 'DrTuvok',
 'Crankgorilla',
 'DrProfessor',
 'DanR-Sousa',
 'BrunoRatesTheMovies',
 'Ashraile',
 'DLochner',
 'Behnam-A',
 'Benslie',
 'AlphaBravoCheesecake',
 'Arcadia1701',
 'Birminghamukengland',
 'Benji-80',
 'Boristhemoggy',
 'Entertainmentsparadise',
 'DeadMansTrousers',
 'Bonnell7',
 'Cinephilia1508',
 'DarthWalker-34381',
 'ChrisBeaken',
 'Deadpool2016',
 'Eric266',
 'Eiriksterminator',
 'Athanatos173',
 'CtlAltDel',
 'Camerenth',
 'AndersSTHLM',
 '77Lynne',
 'BoomerFan',
 'Beni_McBeni',
 'Dev1999',
 'Dr_Mark_ODoherty',
 'Drawmort',
 'Cax019',
 'DrCCat',
 'CriticalOfEverything',
 'C0SSTY',
 'AziziOthmanMY',
 'Apollo15AnnoPianoCatDogSnailAnt',
 'Beachbum2018',
 'DaegT',
 'BBogus',
 'CX78-I',
 'El-Kapitoshka',
 'FattyBoomBatty',
 'Bibiliography',
 'Ark7754',
 'Ashitaka137',
 'Deux_Vega',
 'DJ_Reticul